In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

if os.path.exists('./file/2.item'):
    shutil.rmtree('./file/2.item')

os.makedirs('./file/2.item')

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/1.item_number'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

for file in list_file:
    
    # = = = = = = = = = = = = = = =

    output_correct, output_error = pd.DataFrame(), pd.DataFrame()

    # = = = = = = = = = = = = = = =

    input_ = pd.read_excel(f'./file/1.item_number/{file}',
                           header=0,
                           dtype=str).fillna('')
    
    input_['No'] = input_['No'].astype(int)

    print(f'总数量：{len(input_)}')
    print()

    # = = = = = = = = = = = = = = =


    work = Queue()
    for a in range(len(input_)):
        work.put_nowait(a)

    # = = = = = = = = = = = = = = =

    def crawler():
        global output_correct, output_error

        while not work.empty():
            a = work.get_nowait()

            # = = = = = = = = = = = = = = =

            crawler_status = 'error'

            # = = = = = = = = = = = = = = =

            try:
                b = 0
                while True:
                    b += 1
                    
                    try:
                        resp = requests.get(input_.loc[a, 'Url'],
                                            headers=get_header(ua=False),
                                            proxies=get_proxy(),
                                            timeout=(10, 10))

                        if resp.status_code == 200 or resp.status_code == 404:
                            break
                    except KeyboardInterrupt:
                        break
                    except:
                        continue

                # = = = = = = = = = = = = = = =

                if resp.status_code == 404:
                    raise

                # = = = = = = = = = = = = = = =

                json_ = json.loads(resp.text.split('$MC=(window.$MC||[]).concat(')[1].split(']}]})</script>')[0] + ']}]}')

                # = = = = = = = = = = = = = = =

                if input_.loc[a, 'Url'].startswith('https://www.ebay.com/'):
                    market, country, language, locale = 'EBAY-US', 'US', 'en', 'en-US'
                elif input_.loc[a, 'Url'].startswith('https://www.ebay.de/'):
                    market, country, language, locale = 'EBAY-DE', 'DE', 'de', 'de-DE'
                elif input_.loc[a, 'Url'].startswith('https://www.ebay.co.uk/'):
                    market, country, language, locale = 'EBAY-GB', 'GB', 'en', 'en-GB'
                elif input_.loc[a, 'Url'].startswith('https://www.ebay.com.au/'):
                    market, country, language, locale = 'EBAY-AU', 'AU', 'en', 'en-AU'
                elif input_.loc[a, 'Url'].startswith('https://www.ebay.ca/'):
                    market, country, language, locale = 'EBAY-ENCA', 'CA', 'en', 'en-CA'
                
                json_['o']['g']['marketplaceId'].strip() != market or json_['o']['g']['country'].strip() != country or json_['o']['g']['language'].strip() != language or json_['o']['g']['locale'].strip() != locale or json_['o']['g']['itemId'].strip() != input_.loc[a, 'Item Number']
                
                # = = = = = = = = = = = = = = =

                index_model = -1
                for i in range(len(json_['o']['w'])):
                    if 'model' in json_['o']['w'][i][2]:
                        index_model = i
                        break
                
                # = = = = = = = = = = = = = = =

                if index_model == -1:
                    raise
                
                # = = = = = = = = = = = = = = =

                compatibility = json_['o']['w'][index_model][2]['model']['modules']['COMPATIBILITY_TABLE']['paginatedTable']['pagination']['pages'][-1] if 'COMPATIBILITY_TABLE' in json_['o']['w'][index_model][2]['model']['modules'] else ''
                dict_compatibility = {} if compatibility == '' else {'api': compatibility['action']['URL'],
                                                                     'page': int(compatibility['text'].strip()),
                                                                     'data': {'scopedContext': {'catalogDetails': {'categoryId': compatibility['action']['params']['categoryId'],
                                                                                                                   'itemId': compatibility['action']['params']['itemId'],
                                                                                                                   'marketplaceId': compatibility['action']['params']['marketplaceId'],
                                                                                                                   'sellerName': compatibility['action']['params']['sellerName']}}}}

                # = = = = = = = = = = = = = = =

                dict_specific = {}
                dict_item = json_['o']['w'][index_model][2]['model']['modules']['ABOUT_THIS_ITEM']['sections']['features']['dataItems'] if 'ABOUT_THIS_ITEM' in json_['o']['w'][index_model][2]['model']['modules'] else {}
                for i, (key, value) in zip(range(len(dict_item.items())), dict_item.items()):
                    dict_specific[str(i)] = {value['labels'][0]['textSpans'][0]['text'].strip(): value['values'][0]['textualDisplays'][0]['textSpans'][0]['text'].strip() if 'textualDisplays' in value['values'][0] else value['values'][0]['textSpans'][0]['text'].strip()}

                # = = = = = = = = = = = = = = =

                description_url = json_['o']['w'][index_model][2]['model']['modules']['ITEM_DESCRIPTION_MIN_VIEW_MODULE']['sections'][0]['dataItems'][0]['action']['URL'].strip() if 'dataItems' in json_['o']['w'][index_model][2]['model']['modules']['ITEM_DESCRIPTION_MIN_VIEW_MODULE']['sections'][0] else json_['o']['w'][index_model][2]['model']['modules']['ITEM_DESCRIPTION_MIN_VIEW_MODULE']['sections'][0]['action']['URL'].strip()

                # = = = = = = = = = = = = = = =

                title = json_['o']['w'][index_model][2]['model']['modules']['TITLE']['mainTitle']['textSpans'][0]['text'].strip()

                # = = = = = = = = = = = = = = =

                subtitle = json_['o']['w'][index_model][2]['model']['modules']['TITLE']['subTitle']['textSpans'][0]['text'].strip() if 'subTitle' in json_['o']['w'][index_model][2]['model']['modules']['TITLE'] else ''

                # = = = = = = = = = = = = = = =

                price = json_['o']['w'][index_model][2]['model']['modules']['BUY_BOX']['binModel']['price']['textSpans'][0]['text'].strip() if 'binModel' in json_['o']['w'][index_model][2]['model']['modules']['BUY_BOX'] else json_['o']['w'][0][2]['model']['modules']['BUY_BOX']['bidPrice']['textSpans'][0]['text'].strip()

                # = = = = = = = = = = = = = = =

                available, sold = '', ''
                list_quantity = json_['o']['w'][index_model][2]['model']['modules']['QUANTITY']['availabilitySignal']['textSpans'] if 'QUANTITY' in json_['o']['w'][index_model][2]['model']['modules'] else []
                for quantity in list_quantity:
                    if input_.loc[a, 'Url'].startswith('https://www.ebay.de/'):
                        if 'verfügbar' in quantity['text']:
                            available = quantity['text'].strip()
                        elif 'verkauft' in quantity['text']:
                            sold = quantity['text'].strip()
                    else:
                        if 'available' in quantity['text']:
                            available = quantity['text'].strip()
                        elif 'sold' in quantity['text']:
                            sold = quantity['text'].strip()
            
                # = = = = = = = = = = = = = = =

                dict_src = {}
                list_media = json_['o']['w'][index_model][2]['model']['modules']['PICTURE']['mediaList'] if 'mediaList' in json_['o']['w'][index_model][2]['model']['modules']['PICTURE'] else []
                for i, media in zip(range(len(list_media)), list_media):
                    dict_src[str(i)] = media['image']['originalImg']['URL'].strip()

                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                                         'Item Number': input_.loc[a, 'Item Number'],
                                         'Title': title,
                                         'Subtitle': subtitle,
                                         'Price': price,
                                         'Sold': sold,
                                         'Available': available,
                                         'Picture': '',
                                         'Url': input_.loc[a, 'Url'],
                                         'Description Url': description_url,
                                         'Json_Src': json.dumps(dict_src),
                                         'Json_Specific': json.dumps(dict_specific),
                                         'Json_Compatibility': json.dumps(dict_compatibility)}])

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'
            except:
                output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and work.qsize() == 0 or work.qsize() % 1_000 == 0 and work.qsize() != 0:
                clear_output()

            print(f'''[状态：{crawler_status}，尝试次数：{b}] - {(list_file.index(file) + 1) / len(list_file) * 100:.2f}% - {file.removesuffix('.xlsx')} > {input_.loc[a, 'No']}.{input_.loc[a, 'Item Number']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

    # = = = = = = = = = = = = = = =

    list_task = []
    for _ in range(5):
        task = gevent.spawn(crawler)
        list_task.append(task)
    gevent.joinall(list_task)

    print('输出ing...')
    print()
    if not output_correct.empty:
        output_correct = output_correct.sort_values(by=['No'],
                                                    ascending=[True],
                                                    ignore_index=True)
        output_correct.to_excel(f'''./file/2.item/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if not output_error.empty:
        output_error = output_error.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
        output_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-item_error.xlsx''', index=False)
        print('爬虫存在error')
        print()

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/2.item'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'./file/2.item/{file}',
              f'''./file/2.item/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

总数量：1

[状态：ok，尝试次数：12] - 100.00% - crawler_1-item_error-item_error > 970.204975916441
[剩余数量：0] - [当前时间：18:33:49]

输出ing...



Progress: 100%|██████████████████████████████| 1/1 [00:00<00:00, 3276.80it/s]

Done ~
